In [ ]:
import re

In [ ]:
def tr3(string, lista): # conversão e filtragem
    stringnova=""
    tam=len(string)
    x=0
    while x<tam:
        if(x<tam-1 and str(string[x]) + str(string[x+1]) in lista):
            stringnova+= str(" ")+str(string[x])+str(string[x+1])+str(" ")
            x+=1
        elif(str(string[x]) in lista):
            stringnova+=" "+str(string[x])+" "
        else:
            stringnova+=str(string[x])
        x+=1
    return list(filter(lambda x:x!="",stringnova.split(" ")))

In [ ]:
class Grammar:
    def __init__(self, rules):
        rules = tuple(rules)
        self.rules = tuple(self._parse(rule) for rule in rules)

    def _parse(self, rule):
        return tuple(rule.replace(' ', '').split('->'))
        
    @staticmethod 
    def is_nonterminal(symbol):
        return symbol.isalpha() and symbol.isupper()

    @property
    def nonterminals(self):
        return set([nt for nt, _ in self.rules])
        
    @property
    def terminals(self):       
        a = [
            expression
            for _, expression in self.rules
            if not self.is_nonterminal(expression)
        ]
        
        nt = sorted([nt for nt, _ in self.rules], key=len, reverse=True)
        a = [i for j in [tr3(i, nt) for i in a] for i in j]

        for i in nt: 
            for j in range(0, len(a)):
                if re.search(i, a[j]):
                    a[j] = a[j].replace(i, '')
                a[j] = " ".join(a[j].split())
        
        tmp = []  
        for i in a:
            if i != '' and i not in tmp:
                tmp.append(i)
        a = tmp

        tmp = []
        for i in a:
            if len(i) > 1 and not i.isalpha():
                tmp2 = []
                tmp2.append(list(i))
                tmp2 = [i for j in tmp2 for i in j]
                for j in tmp2:
                    tmp.append(j)
            else:
                tmp.append(i)
        a = tmp
        tmp = [] 
        
        return set(a) 

In [ ]:
def transform2(string,lista):
    listanova=[]
    texto=""
    tam=len(string)
    x = 0
    while x < tam:
        texto +=string[x]
        if(x < tam-1 and texto+string[x+1] in lista):
            listanova.append(texto+string[x+1])
            texto=""
            x +=1
        elif texto in lista: 
            listanova.append(texto)
            texto=""
        x+=1
    return listanova

In [ ]:
def union(first, begins):
    n = len(first)
    first |= begins
    return len(first) != n

In [ ]:
def FirstAndFollow(grammar):
    first = {i: set() for i in grammar.nonterminals}
    first.update((i, {i}) for i in grammar.terminals) 
    follow = {i: set() for i in grammar.nonterminals} 

    lista = sorted((list(grammar.terminals)+list(grammar.nonterminals)), key=len, reverse=True)
    rule = tuple([(i, transform2(j, lista)) for i, j in grammar.rules])
    epsilon = {'ε'}

    while True:
        updated = False
        for nt, expression in rule:
            for symbol in expression:
                updated |= union(first[nt], first[symbol])
                if symbol not in epsilon:
                    break
                else:
                    updated |= union(epsilon, {nt})
            aux = follow[nt]
            for symbol in reversed(expression):
                if symbol in follow:
                    updated |= union(follow[symbol], aux)
                if symbol in epsilon:
                    aux = aux.union(first[symbol])
                else:
                    aux = first[symbol]

        if not updated:
            for chave, valor in follow.items():
                if 'ε' in follow[chave]:
                    follow[chave] = follow[chave] - {'ε'}
            cond1 = False
            cond2 = False
            for i in epsilon:
                if '^' in i:
                    cond1 = True
                if 'ε' in i:
                    cond2 = True

            for i in grammar.nonterminals:
                if '^' in i:
                    first.pop(i)
                    follow.pop(i)
                if cond1 and '^' in i:
                    epsilon.remove(i)

            for i in grammar.terminals:
                first.pop(i)
                if cond2 and 'ε' in i:
                    epsilon.remove(i)
    
            return first, follow, epsilon

In [ ]:
def tratarArq(lista): 
    lista = " ".join(lista.split())
    lista = re.split(r' ', lista)
    return lista

In [ ]:
def lerArquivo(dir):
    arquivo = []
    b = []
    inicio = ""
    fim = ""
    
    with open(dir, "r") as gramatica:
        for line in gramatica:
            arquivo.append(line.replace('Îµ', 'ε').strip().split('\n'))
    arquivo = [i for j in arquivo for i in j]

    for i in range(0, len(arquivo)):
        temp = []
        temp = tratarArq(arquivo[i])
        arquivo[i] = temp
             
    for i in range(0, len(arquivo)):
        tam = len(arquivo[i])
        for j in range(0, tam):    
            if "|" in arquivo[i][j]:
                inicio = arquivo[i][0]+" "+arquivo[i][1]
                fim = arquivo[i][-1]
                suma = inicio+" "+fim
                arquivo.append(tratarArq(suma))
    
    for i in arquivo:
        b.append(" ".join(i[:3]))
    return b

In [ ]:
def gravarArquivo(first, follow, epsilon):
    with open('First.txt', 'w+', encoding="utf-8") as file:
        a = "Epsilon = {}".format(epsilon)
        file.write(a + '\n')
        for i in first:
            a = "Primeiro({}) = {}".format(i, first[i])
            file.write(a + '\n')
    print("\n")
    with open('Follow.txt', 'w+', encoding="utf-8") as file:
        a = ''
        for i in follow:
            a = "Follow({}) = {}".format(i, follow[i])
            file.write(a + '\n')

In [ ]:
def printArquivo(dir):
    with open(dir, "r", encoding="utf-8") as file:
        for line in file:
            print(line)

In [ ]:
def visualizar_gr(lista):
    for i in lista:
        print(i)

In [ ]:
def main():
    dir = "gramaticas"
    gramatica = [dir+'/a.txt', dir+'/b.txt', dir+'/c.txt', dir+'/d.txt', dir+'/e.txt', dir+'/f.txt', dir+'/g.txt']
    escolha = int(input('1 - a.txt\n2 - b.txt\n3 - c.txt\n4 - d.txt\n5 - e.txt\n6 - f.txt\n7 - g.txt\n\nEscolha uma Gramatica: '))
    caminho = gramatica[escolha-1]
    gr = lerArquivo(caminho) 
    visualizar_gr(gr)
    a = Grammar(gr)

    first, follow, epsilon = FirstAndFollow(a)
    gravarArquivo(first, follow, epsilon)
    print('-----------------------------------------------------------------')
    X = True
    while X:
        op = input('1 - Visualizar First\n2 - Visualizar Follow\n3 - Visualizar First e Follow\n\nDigite a Opção: ')
        print('\n---------------------------------------------------------------------\n')
        if op == '1':
            printArquivo('First.txt')
            X = False
        if op == '2':
            printArquivo('Follow.txt')
            X = False
        if op == '3':
            printArquivo('First.txt')
            print('-----------------------------------------------------------------\n')
            printArquivo('Follow.txt')
            X = False

In [ ]:
main()

In [ ]:
# Anotações
# Logica 
# (('E', "TE'"), ("E'", "+TE'"), ('T', "FT'"), ("T'", "*FT'"), ('F', '(E)'), ("E'", 'ε'), ("T'", 'ε'), ('F', 'id'))

# [['E', "TE'"], ["E'", "+TE'"], ['T', "FT'"], ["T'", "*FT'"], ['F', '(E)'], ["E'", 'ε'], ["T'", 'ε'], ['F', 'id']]

# [['E', ["T","E'"]], ["E'", ["+","T","E'"]], ['T', ["F","T'"]], ["T'", ["*","F","T'"]], ['F', ["(","E",")"]], ["E'", ['ε']], ["T'", ['ε']], ['F', ['id']]]

# (('E', ["T","E'"]), ("E'", ["+","T","E'"]), ('T', ["F","T'"]), ("T'", ["*","F","T'"]), ('F', ["(","E",")"]), ("E'", ['ε']), ("T'", ['ε']), ('F', ['id']) )